In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import seaborn as sns
import analysis as ana
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.utils.class_weight import compute_sample_weight
import shap
import datetime
from sklearn.datasets import make_hastie_10_2
import logging
import sys

In [15]:
newphoton_PU200_gen = "/data/data.polcms/cms/debnath/HGCAL/CMSSW_14_0_5/src/shower_shape_studies/samples/photonPU200_newalgogen_filtered.h5"
newphoton_PU200_cl3d = "/data/data.polcms/cms/debnath/HGCAL/CMSSW_14_0_5/src/shower_shape_studies/samples/photonPU200_newalgocl3d_filtered.h5"

In [ ]:
df_gen = pd.read_hdf(newphoton_PU200_gen, key="gen")
df_p016 = pd.read_hdf(newphoton_PU200_cl3d, key="cl3d_p016Tri")
df_p045 = pd.read_hdf(newphoton_PU200_cl3d, key="cl3d_p045Tri")
df_p03  = pd.read_hdf(newphoton_PU200_cl3d, key="cl3d_p03Tri")

ValueError: Dataset(s) incompatible with Pandas data types, not table, or no datasets found in HDF5 file.

In [12]:
newPhoton_PU200 = pd.read_hdf(newphoton_PU200_gen)

In [13]:
newPhoton_PU200

,event,gen_n,gen_eta,gen_phi,gen_pt,gen_energy,gen_pdgid,gen_status,genpart_exphi,genpart_exeta
0,184192,2,-2.223511,-2.927483,51.520218,240.804077,22,1,-2.927484,-2.236255
1,184192,2,2.223511,0.214109,51.520218,240.804077,22,1,0.214110,2.210939
2,184195,2,-2.461290,-1.479612,81.626144,481.808258,22,1,-1.479613,-2.433817
3,184195,2,2.461290,1.661981,81.626144,481.808258,22,1,1.661983,2.489574
4,184202,2,-1.985179,-1.685763,43.618767,161.775589,22,1,-1.685756,-1.999865
...,...,...,...,...,...,...,...,...,...,...
217871,252384,2,-1.806203,1.595164,51.765167,161.806641,22,1,1.595147,-1.805792
217872,252390,2,-2.545979,-2.667971,9.841117,63.151001,22,1,-2.667961,-2.548119
217873,252390,2,2.545979,0.473622,9.841117,63.151001,22,1,0.473612,2.543845
217874,252386,2,-2.422272,1.256119,65.120361,369.888824,22,1,1.256105,-2.441029
